In [1]:
import torchvision
from torchvision import transforms
import torch
from torch import nn
from torch.utils.data import DataLoader
import os

In [41]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
print(torch.cuda.is_available())
device = "cuda:0"
name = "K_mnist"

True


In [42]:
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0_K_mnist.pth')
estado_fase_1 = torch.load('Fase_1_K_mnist.pth')


In [43]:
print(estado_fase_1["linear_relu_stack.4.weight"])
print(estado_fase_1["linear_relu_stack.4.bias"])

tensor([[ 0.0467, -0.0436, -0.0520,  ..., -0.0072,  0.0159, -0.0434],
        [-0.0193, -0.0183, -0.0475,  ...,  0.0255,  0.0287, -0.0241],
        [ 0.0035, -0.0295,  0.0244,  ..., -0.0839,  0.0592, -0.0232],
        ...,
        [ 0.0342,  0.0853, -0.0248,  ..., -0.0501, -0.0051,  0.0596],
        [-0.0463,  0.0192,  0.0688,  ..., -0.0521, -0.0098, -0.0415],
        [ 0.0554, -0.0577, -0.0438,  ...,  0.0010, -0.0382, -0.0468]],
       device='cuda:0')
tensor([-0.0302,  0.1261,  0.1129,  0.1251, -0.1550,  0.0587, -0.0835, -0.1828,
         0.0273, -0.0179], device='cuda:0')


## Change f_0(5-9) to f_1(5-9) MODEL 1

In [17]:

estado_fase_1["linear_relu_stack.4.weight"][5:10, :] = estado_fase_0["linear_relu_stack.4.weight"][5:10, :]
estado_fase_1["linear_relu_stack.4.bias"][5:10] = estado_fase_0["linear_relu_stack.4.bias"][5:10]

txt = f"logs/preditions_combination_classifier_f1_0_4_f0_5_9_F1_{name}.txt"

In [18]:

model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_1)

<All keys matched successfully>

## Change f_0(0-4) to f_1(0-4) MODEL 1

In [22]:
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0_K_mnist.pth')
estado_fase_1 = torch.load('Fase_1_K_mnist.pth')


In [23]:

# Transferir el clasificador de Fase_0 a Fase_1
a, b = 0, 5
estado_fase_1["linear_relu_stack.4.weight"][a:b, :] = estado_fase_0["linear_relu_stack.4.weight"][a:b, :]
estado_fase_1["linear_relu_stack.4.bias"][a:b] = estado_fase_0["linear_relu_stack.4.bias"][a:b]

txt = f"logs/preditions_combination_classifier_f0_0_4_f1_5_9_F1_{name}.txt"

In [24]:

model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_1)

<All keys matched successfully>

## Change f_1(0-4) to f_0(0,4) MODEL 0

In [44]:
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0_K_mnist.pth')
estado_fase_1 = torch.load('Fase_1_K_mnist.pth')

In [45]:

a, b = 0, 5
estado_fase_0["linear_relu_stack.4.weight"][a:b, :] = estado_fase_1["linear_relu_stack.4.weight"][a:b, :]
estado_fase_0["linear_relu_stack.4.bias"][a:b] = estado_fase_1["linear_relu_stack.4.bias"][a:b]

txt = f"logs/preditions_combination_classifier_f1_0_4_f0_5_9_F0_{name}.txt"

In [46]:
model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_0)

<All keys matched successfully>

## Change f_1(5-9) to f_1(5-9) MODEL 0

In [48]:
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0_K_mnist.pth')
estado_fase_1 = torch.load('Fase_1_K_mnist.pth')

In [49]:

# Transferir el clasificador de Fase_0 a Fase_1
a, b = 5, 10
estado_fase_0["linear_relu_stack.4.weight"][a:b, :] = estado_fase_1["linear_relu_stack.4.weight"][a:b, :]
estado_fase_0["linear_relu_stack.4.bias"][a:b] = estado_fase_1["linear_relu_stack.4.bias"][a:b]

txt = f"logs/preditions_combination_classifier_f0_0_4_f1_5_9_F0_{name}.txt"

In [50]:

model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_0)

<All keys matched successfully>

### Evaluación

In [34]:
def test_loop(dataloader, model, loss_fn, txt):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    tarjet_prediction = []  # Lista para almacenar las etiquetas reales y predicciones

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # Guardar la etiqueta real y la predicción en la lista de tuplas
            tarjet_prediction.extend(list(zip(y.cpu().numpy(), pred.argmax(1).cpu().numpy())))

    test_loss /= num_batches
    correct /= size
    log_accuracy_loss.append((100*correct, test_loss))
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

    with open(f'{txt}', 'w') as archivo:
            # Escribe el valor de la variable en el archivo
            archivo.write(str(tarjet_prediction))
    print(f'El valor prediciones se ha guardado en el archivo.txt')
    

In [35]:
eval_data = torchvision.datasets.KMNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
eval_dataloader = DataLoader(eval_data, batch_size=10000, shuffle=True)
print("Se cargaron los datos correctamente")

Se cargaron los datos correctamente


In [51]:
learning_rate = 1e-3
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_classifier.parameters(), lr=learning_rate)
epochs = 1
log_accuracy_loss = []
test_loop(eval_dataloader, model_classifier, loss_fn, txt)

Test Error: 
 Accuracy: 40.6%, Avg loss: 2.919341 

El valor prediciones se ha guardado en el archivo.txt
